# Conversor de Moedas (Notebook)

Este notebook contém implementações mínimas para obter taxas de câmbio e realizar conversões.
As funções foram adaptadas a partir dos arquivos `servidor.py` e `cliente.py` do projeto.

Usos: importar as funções deste notebook em outros scripts ou executar as células aqui para testar localmente.

In [ ]:
# Bibliotecas necessárias
import requests
from datetime import datetime

print('Bibliotecas carregadas')

In [ ]:
def get_fallback_rates():
    """Retorna um dicionário de taxas fallback (USD base)."""
    return {
        'USD': 1.0,
        'BRL': 5.12,
        'EUR': 0.92,
        'GBP': 0.79,
        'JPY': 149.50,
    }

In [ ]:
def get_exchange_rates():
    """Tenta obter taxas da ExchangeRate-API (base USD). Em falha retorna fallback."""
    try:
        url = 'https://api.exchangerate-api.com/v4/latest/USD'
        resp = requests.get(url, timeout=5)
        if resp.status_code == 200:
            data = resp.json()
            rates = data.get('rates', {})
            return rates
    except Exception as e:
        print('Erro ao buscar API:', e)

    # Fallback
    print('Usando taxas fallback')
    return get_fallback_rates()

In [ ]:
def convert_currency(from_curr, to_curr, amount, rates=None):
    """Converte `amount` de `from_curr` para `to_curr` usando `rates` (base USD).
    Retorna um dicionário com resultado, taxa e metadados."""
    if rates is None:
        rates = get_exchange_rates()

    from_curr = from_curr.upper()
    to_curr = to_curr.upper()

    if from_curr not in rates:
        raise ValueError(f'Moeda {from_curr} não suportada')
    if to_curr not in rates:
        raise ValueError(f'Moeda {to_curr} não suportada')

    amount = float(amount)
    if amount <= 0:
        raise ValueError('Valor deve ser maior que zero')

    # Converte via USD (base das taxas)
    usd_amount = amount / rates[from_curr]
    result = usd_amount * rates[to_curr]
    rate = rates[to_curr] / rates[from_curr]

    return {
        'from': from_curr,
        'to': to_curr,
        'amount': amount,
        'result': result,
        'rate': rate,
        'fetched_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

In [ ]:
# Exemplo rápido de uso
rates = get_exchange_rates()
print('Moedas carregadas (exemplo):', ', '.join(list(rates.keys())[:10]))

# Converter 100 USD para BRL
try:
    res = convert_currency('USD', 'BRL', 100, rates)
    print(f"{res['amount']} {res['from']} => {res['result']:.2f} {res['to']} (taxa: {res['rate']:.6f}) fetched: {res['fetched_at']}")
except Exception as e:
    print('Erro na conversão:', e)

## Observações
- Este notebook faz chamadas HTTP; se sua máquina estiver offline as taxas fallback serão usadas.
- Para integrar com `cliente.py`/`servidor.py` você pode importar as funções `get_exchange_rates` e `convert_currency`.
- Caso queira testar o servidor localmente, execute `servidor.py` em uma janela de terminal e depois `cliente.py` em outra.